[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Hawksight-AI/semantica/blob/main/cookbook/use_cases/biomedical/01_Drug_Discovery_Pipeline.ipynb)

# Drug Discovery Pipeline - Vector Similarity Search

## Overview

This notebook demonstrates a **complete drug discovery pipeline** using Semantica's modular architecture. We'll use individual modules directly to build a comprehensive system for drug-target interaction prediction using vector similarity search and knowledge graphs.

### Key Features

- **Modular Architecture**: Uses Semantica modules directly (`NERExtractor`, `GraphBuilder`, `EmbeddingGenerator`, `VectorStore`)
- **Multiple Data Sources**: Ingests from 15+ PubMed RSS feeds, preprint servers, and journal feeds
- **Vector Similarity Search**: Emphasizes embeddings and vector similarity for drug-target interaction prediction
- **Entity Extraction**: Extracts drug compounds, proteins, targets, enzymes, and receptors
- **Knowledge Graph**: Builds structured drug-target relationship graphs
- **GraphRAG**: Hybrid vector + graph retrieval for enhanced querying

### What You'll Learn

- How to use Semantica modules directly (avoiding the core orchestrator)
- How to ingest biomedical data from multiple sources
- How to extract entities using `NERExtractor`
- How to extract relationships using `RelationExtractor`
- How to generate embeddings with `EmbeddingGenerator`
- How to build knowledge graphs with `GraphBuilder`
- How to perform similarity search with `VectorStore`
- How to use GraphRAG with `AgentContext` for hybrid retrieval

### Pipeline Flow

```mermaid
graph LR
    A[Data Ingestion] --> B[Text Processing]
    B --> C[Entity Extraction]
    C --> D[Relationship Extraction]
    D --> E[Deduplication]
    E --> F[Embedding Generation]
    F --> G[Vector Store]
    G --> H[Knowledge Graph]
    H --> I[Similarity Search]
    H --> J[GraphRAG Queries]
    I --> K[Visualization]
    J --> K
```

### Data Sources

**PubMed RSS Feeds:**
- Drug Discovery, Drug Target Interaction, Pharmacokinetics, Pharmacodynamics
- Clinical Trials, Protein Targets, Drug Repurposing, Molecular Docking
- ADME, Drug Metabolism, Drug Safety, Precision Medicine
- Biomarkers, Drug Resistance, Combinatorial Therapy

**Preprint Servers:**
- BioRxiv (Pharmacology & Toxicology, Drug Discovery)
- MedRxiv (Clinical Trials)
- ChemRxiv

**Journal RSS Feeds:**
- Nature (Drug Discovery, Pharmacology)
- Science Translational Medicine
- Cell Chemical Biology
- Journal of Medicinal Chemistry
- Drug Discovery Today
- Trends in Pharmacological Sciences


---


## Installation

Install Semantica and required dependencies:


In [1]:
%pip install -qU semantica networkx matplotlib plotly pandas faiss-cpu beautifulsoup4 groq sentence-transformers scikit-learn


Note: you may need to restart the kernel to use updated packages.


## Configuration & Setup

Set up environment variables and configuration constants.


In [2]:
import os

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY", "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")


In [3]:
EMBEDDING_DIMENSION = 384
EMBEDDING_MODEL = "all-MiniLM-L6-v2"
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200


## Ingesting Biomedical Data from Multiple Sources

Ingest data from comprehensive biomedical sources including PubMed RSS feeds, preprint servers, and journal feeds.


In [4]:
from semantica.ingest import FeedIngestor, FileIngestor
import os
from contextlib import redirect_stderr
from io import StringIO

os.makedirs("data", exist_ok=True)

feed_sources = [
    # Nature Feeds
    ("Nature - Drug Discovery", "https://www.nature.com/subjects/drug-discovery.rss"),
    ("Nature - Pharmacology", "https://www.nature.com/subjects/pharmacology.rss"),
    ("Nature Reviews Drug Discovery", "https://www.nature.com/nrd.rss"),
    
    # FDA & Government Sources
    ("FDA MedWatch", "https://www.fda.gov/AboutFDA/ContactFDA/StayInformed/RSSFeeds/MedWatch/rss.xml"),
    ("NCI News", "https://www.cancer.gov/syndication/rss"),
    
    # Drug Information & News
    ("Drugs.com - MedNews", "https://www.drugs.com/rss/mednews.xml"),
    ("Drugs.com - FDA Alerts", "https://www.drugs.com/rss/fda-alerts.xml"),
    ("Drugs.com - Clinical Trials", "https://www.drugs.com/rss/clinical-trials.xml"),
    
    # Medical News
    ("Labroots Health & Medicine", "http://www.labroots.com/rss/trending/health-and-medicine"),
    ("Biology News Net", "https://www.biologynews.net/rss.php"),
    
    # Open Access Journals
    ("PLOS ONE - Medicine", "https://journals.plos.org/plosone/feed/atom"),
    ("PLOS Biology", "https://journals.plos.org/plosbiology/feed/atom"),
    ("PLOS Medicine", "https://journals.plos.org/plosmedicine/feed/atom"),
    
    # Preprint Servers
    ("arXiv - q-bio", "http://arxiv.org/rss/q-bio"),
    ("arXiv - q-bio.BM", "http://arxiv.org/rss/q-bio.BM"),
]

feed_ingestor = FeedIngestor()
all_documents = []

print(f"Ingesting from {len(feed_sources)} feed sources...")
for i, (feed_name, feed_url) in enumerate(feed_sources, 1):
    try:
        with redirect_stderr(StringIO()):
            feed_data = feed_ingestor.ingest_feed(feed_url, validate=False)
        
        feed_count = 0
        for item in feed_data.items:
            if not item.content:
                item.content = item.description or item.title or ""
            if item.content:
                if not hasattr(item, 'metadata'):
                    item.metadata = {}
                item.metadata['source'] = feed_name
                all_documents.append(item)
                feed_count += 1
        
        if feed_count > 0:
            print(f"  [{i}/{len(feed_sources)}] {feed_name}: {feed_count} documents")
    except Exception:
        continue

if not all_documents:
    sample_drug_data = """
    Aspirin (acetylsalicylic acid) is a medication used to reduce pain, fever, or inflammation. 
    It targets cyclooxygenase enzymes COX-1 and COX-2. Aspirin is commonly used for cardiovascular protection.
    Ibuprofen is a nonsteroidal anti-inflammatory drug (NSAID) that targets COX-1 and COX-2 enzymes.
    Metformin is an antidiabetic medication that targets AMP-activated protein kinase (AMPK).
    Insulin targets the insulin receptor (INSR) to regulate glucose metabolism.
    Warfarin is an anticoagulant that targets vitamin K epoxide reductase complex subunit 1 (VKORC1).
    Atorvastatin is a statin medication that targets HMG-CoA reductase.
    """
    
    with open("data/sample_drugs.txt", "w") as f:
        f.write(sample_drug_data)
    
    file_ingestor = FileIngestor()
    all_documents = file_ingestor.ingest("data/sample_drugs.txt")

documents = all_documents
print(f"Ingested {len(documents)} documents")


Ingesting from 15 feed sources...


Status,Action,Module,Submodule,File,Time
✅,Semantica is ingesting,📥 ingest,FeedIngestor,q-bio.BM,1.08s
✅,Semantica is normalizing,🔧 normalize,TextNormalizer,-,0.00s
🔄,Semantica is extracting,🎯 semantic_extract,NERExtractor,-,0.00s
✅,Semantica is extracting,🎯 semantic_extract,RelationExtractor,-,0.80s
✅,Semantica is resolving,⚠️ conflicts,ConflictResolver,-,0.00s
🔄,Semantica is building,🧠 kg,EntityResolver,-,1504.08s
🔄,Semantica is deduplicating,🔄 deduplication,DuplicateDetector,-,1504.06s
🔄,Semantica is deduplicating,🔄 deduplication,SimilarityCalculator,-,1480.00s
✅,Semantica is indexing,📊 vector_store,VectorStore,-,0.01s
🔄,Semantica is building,🧠 kg,GraphBuilder,-,927.50s


  [1/15] Nature - Drug Discovery: 30 documents
  [2/15] Nature - Pharmacology: 30 documents
  [9/15] Labroots Health & Medicine: 30 documents
  [11/15] PLOS ONE - Medicine: 30 documents
  [12/15] PLOS Biology: 30 documents
  [13/15] PLOS Medicine: 30 documents
Ingested 180 documents


## Normalizing and Chunking Documents

Clean and normalize text, then split into chunks using entity-aware chunking to preserve drug/protein entity boundaries.


In [5]:
from semantica.normalize import TextNormalizer
from semantica.split import TextSplitter

normalizer = TextNormalizer()
splitter = TextSplitter(
    method="entity_aware",
    ner_method="spacy",
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)

print(f"Normalizing {len(documents)} documents...")
normalized_documents = []
for i, doc in enumerate(documents, 1):
    normalized_text = normalizer.normalize(
        doc.content if hasattr(doc, 'content') else str(doc),
        clean_html=True,
        normalize_entities=True,
        remove_extra_whitespace=True,
        lowercase=False
    )
    normalized_documents.append(normalized_text)
    if i % 50 == 0 or i == len(documents):
        print(f"  Normalized {i}/{len(documents)} documents...")

print(f"Chunking {len(normalized_documents)} documents...")
chunked_documents = []
for i, doc_text in enumerate(normalized_documents, 1):
    try:
        with redirect_stderr(StringIO()):
            chunks = splitter.split(doc_text)
        chunked_documents.extend(chunks)
    except Exception:
        simple_splitter = TextSplitter(method="recursive", chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
        chunks = simple_splitter.split(doc_text)
        chunked_documents.extend(chunks)
    if i % 50 == 0 or i == len(normalized_documents):
        print(f"  Chunked {i}/{len(normalized_documents)} documents ({len(chunked_documents)} chunks so far)")

print(f"Created {len(chunked_documents)} chunks from {len(normalized_documents)} documents")




Normalizing 180 documents...
  Normalized 50/180 documents...
  Normalized 100/180 documents...
  Normalized 150/180 documents...
  Normalized 180/180 documents...
Chunking 180 documents...
  Chunked 50/180 documents (50 chunks so far)
  Chunked 100/180 documents (101 chunks so far)
  Chunked 150/180 documents (194 chunks so far)
  Chunked 180/180 documents (250 chunks so far)
Created 250 chunks from 180 documents


In [6]:
from semantica.semantic_extract import NERExtractor

# Using spaCy ML method (similar to NER cell)
entity_extractor = NERExtractor(method="ml", model="en_core_web_sm")

all_entities = []
print(f"Extracting entities from {len(chunked_documents)} chunks...")

for i, chunk in enumerate(chunked_documents, 1):
    chunk_text = chunk.text if hasattr(chunk, 'text') else str(chunk)
    try:
        entities = entity_extractor.extract_entities(chunk_text)
        all_entities.extend(entities)
    except Exception:
        continue
    
    if i % 20 == 0 or i == len(chunked_documents):
        print(f"  Processed {i}/{len(chunked_documents)} chunks ({len(all_entities)} entities found)")

# Filter entities - spaCy returns standard types (PERSON, ORG, PRODUCT, etc.)
# Map to biomedical categories based on context
drugs = [e for e in all_entities if e.label == "PRODUCT" or (e.label == "ORG" and any(kw in e.text.lower() for kw in ["drug", "pharma", "medication"]))]
proteins = [e for e in all_entities if e.label == "ORG" or (e.label == "PRODUCT" and any(kw in e.text.lower() for kw in ["protein", "enzyme", "receptor", "kinase", "target"]))]

print(f"Extracted {len(drugs)} drugs and {len(proteins)} proteins")


Extracting entities from 250 chunks...
  Processed 20/250 chunks (11 entities found)
  Processed 40/250 chunks (27 entities found)
  Processed 60/250 chunks (47 entities found)
  Processed 80/250 chunks (64 entities found)
  Processed 100/250 chunks (154 entities found)
  Processed 120/250 chunks (312 entities found)
  Processed 140/250 chunks (502 entities found)
  Processed 160/250 chunks (669 entities found)
  Processed 180/250 chunks (873 entities found)
  Processed 200/250 chunks (1105 entities found)
  Processed 220/250 chunks (1478 entities found)
  Processed 240/250 chunks (1967 entities found)
  Processed 250/250 chunks (2195 entities found)
Extracted 26 drugs and 509 proteins


## Extracting Drug-Target Relationships

Extract relationships between drugs and proteins to understand drug-target interactions.


In [7]:
from semantica.semantic_extract import RelationExtractor

# Using spaCy dependency parsing (similar to NER cell)
relation_extractor = RelationExtractor(method="dependency", model="en_core_web_sm")

all_relationships = []
print(f"Extracting relationships from {len(chunked_documents)} chunks...")

for i, chunk in enumerate(chunked_documents, 1):
    chunk_text = chunk.text if hasattr(chunk, 'text') else str(chunk)
    try:
        relationships = relation_extractor.extract_relations(
            chunk_text,
            entities=all_entities,
            relation_types=["targets", "inhibits", "activates", "binds_to", "interacts_with"]
        )
        all_relationships.extend(relationships)
    except Exception:
        continue
    
    if i % 20 == 0 or i == len(chunked_documents):
        print(f"  Processed {i}/{len(chunked_documents)} chunks ({len(all_relationships)} relationships found)")

print(f"Extracted {len(all_relationships)} relationships")


Extracting relationships from 250 chunks...
  Processed 20/250 chunks (25 relationships found)
  Processed 40/250 chunks (43 relationships found)
  Processed 60/250 chunks (63 relationships found)
  Processed 80/250 chunks (109 relationships found)
  Processed 100/250 chunks (155 relationships found)
  Processed 120/250 chunks (365 relationships found)
  Processed 140/250 chunks (577 relationships found)
  Processed 160/250 chunks (773 relationships found)
  Processed 180/250 chunks (1018 relationships found)
  Processed 200/250 chunks (1251 relationships found)
  Processed 220/250 chunks (1478 relationships found)
  Processed 240/250 chunks (1730 relationships found)
  Processed 250/250 chunks (1848 relationships found)
Extracted 1848 relationships


## Resolving Duplicate Entities

Detect and merge duplicate entities to ensure data quality and consistency.


## Conflict Detection and Resolution

Detect and resolve conflicts in drug-target relationships from multiple research sources.

- **Detection Method**: Relationship conflict detection identifies discrepancies in drug-target interactions across sources
- **Resolution Strategy**: Credibility-weighted resolution prioritizes higher-credibility sources (e.g., Nature journals over arXiv preprints)
- **Use Case**: Handles conflicting information when multiple sources report different drug-target relationships


In [11]:
from semantica.conflicts import ConflictDetector, ConflictResolver

# Initialize with best strategies for drug discovery
detector = ConflictDetector()
resolver = ConflictResolver(default_strategy="credibility_weighted")

# Convert relationships to format expected by detector
relationships = [
    {
        "id": f"{rel.subject.text}_{rel.object.text}_{rel.predicate}",
        "source_id": rel.subject.text,
        "target_id": rel.object.text,
        "type": rel.predicate,
        "confidence": rel.confidence,
        "properties": rel.metadata,
        "source": rel.metadata.get("source", "unknown") if hasattr(rel, 'metadata') else "unknown"
    }
    for rel in all_relationships if hasattr(rel, 'subject')
]

# Detect relationship conflicts
conflicts = detector.detect_relationship_conflicts(relationships)
print(f"Detected {len(conflicts)} relationship conflicts from {len(relationships)} relationships")

# Resolve conflicts using credibility-weighted strategy (best for scientific sources)
if conflicts:
    resolved = resolver.resolve_conflicts(conflicts, strategy="credibility_weighted")
    print(f"Resolved {len(resolved)} conflicts using {resolver.default_strategy.value} strategy")


Detected 54 relationship conflicts from 1848 relationships


Resolved 54 conflicts using credibility_weighted strategy


## Generating Vector Embeddings

Generate embeddings for drugs and proteins to enable similarity search.


In [12]:
from semantica.embeddings import EmbeddingGenerator
from semantica.vector_store import VectorStore

embedding_gen = EmbeddingGenerator(
    provider="sentence_transformers",
    model=EMBEDDING_MODEL
)

vector_store = VectorStore(backend="faiss", dimension=EMBEDDING_DIMENSION)

print(f"Generating embeddings for {len(drugs)} drugs and {len(proteins)} proteins...")
drug_texts = [d.text for d in drugs]
drug_embeddings = embedding_gen.generate_embeddings(drug_texts)

protein_texts = [p.text for p in proteins]
protein_embeddings = embedding_gen.generate_embeddings(protein_texts)

print(f"Generated {len(drug_embeddings)} drug embeddings and {len(protein_embeddings)} protein embeddings")


fastembed not available. Install with: pip install fastembed. Using fallback embedding method.
fastembed not available. Install with: pip install fastembed. Using fallback embedding method.


Generating embeddings for 26 drugs and 509 proteins...
Generated 26 drug embeddings and 509 protein embeddings


## Populating Vector Database

Store drug and protein embeddings in the vector database with metadata for efficient similarity search.


In [13]:
print(f"Storing {len(drug_embeddings)} drug vectors and {len(protein_embeddings)} protein vectors...")
drug_ids = vector_store.store_vectors(
    vectors=drug_embeddings,
    metadata=[{"type": "drug", "name": d.text, "label": d.label} for d in drugs]
)

protein_ids = vector_store.store_vectors(
    vectors=protein_embeddings,
    metadata=[{"type": "protein", "name": p.text, "label": p.label} for p in proteins]
)

print(f"Stored {len(drug_ids)} drug vectors and {len(protein_ids)} protein vectors")


Storing 26 drug vectors and 509 protein vectors...
Stored 26 drug vectors and 509 protein vectors


## Building Drug-Target Knowledge Graph

Construct a knowledge graph from extracted entities and relationships to enable graph-based reasoning.


In [ ]:
from semantica.kg import GraphBuilder

# GraphBuilder automatically handles Entity and Relation objects
graph_builder = GraphBuilder(
    merge_entities=True,
    resolve_conflicts=True,
    entity_resolution_strategy="fuzzy"
)

# Pass entities and relationships directly - GraphBuilder handles conversion
kg = graph_builder.build({
    "entities": all_entities,
    "relationships": all_relationships
})

entities_count = len(kg.get('entities', []))
relationships_count = len(kg.get('relationships', []))
print(f"Graph: {entities_count} entities, {relationships_count} relationships")


Building knowledge graph...


## Finding Similar Drugs via Vector Search

Use vector similarity search to find drugs similar to a query drug based on their embeddings.


In [ ]:
query_drug = "Aspirin"
query_embedding = embedding_gen.generate_embeddings([query_drug])[0]
similar_drugs = vector_store.search_vectors(query_embedding, k=5)

print(f"Drugs similar to '{query_drug}':")
for i, result in enumerate(similar_drugs, 1):
    print(f"{i}. {result['metadata'].get('name', 'Unknown')} (similarity: {result['score']:.3f})")


## GraphRAG: Hybrid Vector + Graph Retrieval

Use GraphRAG to combine vector similarity search with knowledge graph traversal for enhanced retrieval and reasoning.


In [ ]:
from semantica.context import AgentContext

context = AgentContext(vector_store=vector_store, knowledge_graph=kg)

query = "What drugs target COX enzymes?"
results = context.retrieve(
    query,
    max_results=10,
    use_graph=True,
    expand_graph=True,
    include_entities=True,
    include_relationships=True
)

print(f"GraphRAG query: '{query}'")
print(f"\nRetrieved {len(results)} results:\n")
for i, result in enumerate(results[:5], 1):
    print(f"{i}. Score: {result.get('score', 0):.3f}")
    print(f"   Content: {result.get('content', '')[:200]}...")
    if result.get('related_entities'):
        print(f"   Related entities: {len(result['related_entities'])}")
    print()


## Visualizing the Knowledge Graph

Generate an interactive visualization of the drug-target knowledge graph.


In [ ]:
from semantica.visualization import KGVisualizer

visualizer = KGVisualizer()
visualizer.visualize(
    kg,
    output_path="drug_target_kg.html",
    layout="spring",
    node_size=20
)

print("Visualization saved to drug_target_kg.html")


## Exporting Results

Export the knowledge graph to various formats for further analysis or integration with other tools.


In [ ]:
from semantica.export import GraphExporter

exporter = GraphExporter()
exporter.export(kg, output_path="drug_target_kg.json", format="json")
exporter.export(kg, output_path="drug_target_kg.graphml", format="graphml")

print("Exported knowledge graph to JSON and GraphML formats")
